In [1]:
from sklearn import metrics
import processing
import time
import os
from tqdm import tqdm
from torch.optim import Adam
from torch import nn
import numpy as np
import sys
sys.path.append("D:/Experiment")
import torch
from MyKu import processing
from MyKu import MyXLM_Base
import pandas as pd

In [2]:

# 训练准备阶段，设置超参数和全局变量
file_name = 'readme.md'
batch_size = 32
num_epoch = 1  # 训练轮次
check_step = 1  # 用以训练中途对模型进行检验：每check_step个epoch进行一次测试和保存模型

learning_rate = 1e-5  # 优化器的学习率

# 获取训练、测试数据、分类类别总数
en_train_data, en_test_data, de_train_data, de_test_data, hi_train_data, hi_test_data = processing.load_hasoc2020()
categories = 2

en_train_iter, en_test_iter = MyXLM_Base.load_xlm_data(en_train_data, en_test_data, batch_size)
de_train_iter, de_test_iter = MyXLM_Base.load_xlm_data(de_train_data, de_test_data, batch_size)
hi_train_iter, hi_test_iter = MyXLM_Base.load_xlm_data(hi_train_data, hi_test_data, batch_size)
#固定写法，可以牢记，cuda代表Gpu
# torch.cuda.is_available()可以查看当前Gpu是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型，因为这里是英文数据集，需要用在英文上的预训练模型：bert-base-uncased
# uncased指该预训练模型对应的词表不区分字母的大小写
# 详情可了解：https://huggingface.co/bert-base-uncased
pretrained_model_name = 'xlm-roberta-base'
# 创建模型 BertSST2Model
model = MyXLM_Base.MyXlmModel(
    categories, pretrained_model_name)
# 固定写法，将模型加载到device上，
# 如果是GPU上运行，此时可以观察到GPU的显存增加
model.to(device)

# 训练过程
# Adam是最近较为常用的优化器，详情可查看：https://www.jianshu.com/p/aebcaf8af76e
optimizer = Adam(model.parameters(), learning_rate)  # 使用Adam优化器
loss = nn.CrossEntropyLoss()  # 使用crossentropy作为二分类任务的损失函数

# 记录当前训练时间，用以记录日志和存储
timestamp = time.strftime("%m_%d_%H_%M", time.localtime())


In [3]:
# TaskA

file_name = 'readme.md'
model_save_path = 'D:/Experiment_models_save/twitter-xlm-roberta-base-sentiment/hasoc2020/'
name = 'taskA.pth'
en_temp_best = 0.89
de_temp_best = 0.85
hi_temp_best = 0.73
for epoch in range(1, num_epoch + 1):
    MyXLM_Base.train(model, en_train_iter, device, optimizer, loss, epoch)
    # MyXLM_Base.train(model, de_train_iter, device, optimizer, loss, epoch)
    # MyXLM_Base.train(model, hi_train_iter, device, optimizer, loss, epoch)
    en_acc_score = MyXLM_Base.test(model, en_test_iter, device, epoch, file_name)
    # de_acc_score = MyXLM_Base.test(model, de_test_iter, device, epoch, file_name)
    # hi_acc_score = MyXLM_Base.test(model, hi_test_iter, device, epoch, file_name)
    print('\n\n')
    # if en_acc_score > en_temp_best and de_acc_score > de_temp_best and hi_acc_score > hi_temp_best:
    #     en_temp_best, de_temp_best, hi_temp_best = en_acc_score, de_acc_score, hi_acc_score
    #     MyXLM_Base.save_pretrained(model, model_save_path, name)
    #     print(f'best en_acc_socre : {en_acc_score}, best de_acc_score : {de_acc_score}, best hi_acc_score : {hi_acc_score}')
print(f'best en_acc_socre : {en_temp_best}, best de_acc_score : {de_temp_best}, best hi_acc_score : {hi_temp_best}')


Testing: 100%|██████████| 1592/1592 [00:17<00:00, 92.91it/s]

[[753  32]
 [178 629]]
              precision    recall  f1-score   support

           0       0.81      0.96      0.88       785
           1       0.95      0.78      0.86       807

    accuracy                           0.87      1592
   macro avg       0.88      0.87      0.87      1592
weighted avg       0.88      0.87      0.87      1592

Acc : 0.8680904522613065	 F1: 0.8672853032526058



best en_acc_socre : 0.89, best de_acc_score : 0.85, best hi_acc_score : 0.73


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
tokenizer

PreTrainedTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [15]:
from bertviz import head_view


def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))
sentence = ['you are a bitch!']
# 'you are like a rotten cockroach!' 
inputs = tokenizer(
    sentence, return_tensors='pt')
inputs.to(device)
print(inputs)
inputs_ids = inputs['input_ids']


{'input_ids': tensor([[   0,  398,  621,   10, 4785,  206,   38,    2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [21]:
inputs_ids = inputs['input_ids']
input_id_list = inputs_ids[0].tolist()
input_id_list
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

tokens

['<s>', '▁you', '▁are', '▁a', '▁bit', 'ch', '!', '</s>']

In [22]:
output = model(inputs)
attention = output[2][0]
len(attention.shape)
# call_html()
head_view(attention, tokens)

# layer = nn.Linear(768, 2).to(device)
# res = layer(attention)
# res = np.argmax(pooler_output, axis=1)
# res
# layer = nn.Linear(768, 2).to(device)
# res = layer(pooler_output)
# ress = layer(output[1])
# gete = np.argmax(res[0].cpu().detach().numpy(), axis=1)
# print(gete)
# gett = np.argmax(ress[0].cpu().detach().numpy())
# print(gett)



# gete2 = np.argmax(res[1].cpu().detach().numpy(), axis=1)
# print(gete2)
# gett2 = np.argmax(ress[1].cpu().detach().numpy())

ValueError: The attention tensor does not have the correct number of dimensions. Make sure you set output_attentions=True when initializing your model.

In [ ]:
# print('\033[1;31m' + 'hello word!' + '\033[0m')
for i in range(1, len(words)):
    if words[i][0] != '▁' and words[i-1][0] == '▁' and gete[i-1] == 1:
        gete[i] = 1

for i in range(1, len(words2)):
    if words2[i][0] != '▁' and words2[i-1][0] == '▁' and gete2[i-1] == 1:
        gete2[i] = 1
print(gete)
for index in range(1,  7):
    word = words[index]
    word = ' ' + word[1:] if word[0] == '▁' else word
    if gete[index] == 1:
        print('\033[1;31m' + word + '\033[0m', end='')
    else:
        print(word, end='')
print()
print(gete2)
for index in range(1, len(gete2) - 1):
    word = words2[index]
    word = ' ' + word[1:] if word[0] == '▁' else word
    if gete2[index] == 1:
        print('\033[1;31m' + word + '\033[0m', end='')
    else:
        print(word, end='')
